In [1]:
import pandas as pd

In [2]:
print("hello")

hello


In [1]:
!pip3 install torch 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 877.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 993.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.7 MB/s eta 0:00:0000:0100:01m


In [3]:
# test if installation is successful
import transformers
from transformers import pipeline
print(pipeline('sentiment-analysis')('we love you'))

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998704195022583}]


In [5]:
#pipelines, the highest-level entity of the transformers framework
MODEL_NAME = 'gpt2'
pipe = pipeline(task='text-generation', model=MODEL_NAME, device='cpu')

config.json: 100%|██████████| 665/665 [00:00<00:00, 3.36MB/s]
model.safetensors: 100%|██████████| 548M/548M [00:04<00:00, 127MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 225kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.38MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 43.4MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 1.46MB/s]


In [6]:
print(pipe('The elf queen'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The elf queen\n\n"Majolotca."\n\nHe paused at that mark before he could finish. "How long would you go through like this?"\n\nHe looked at the old man standing in front of him.\n\n'}]


In [ ]:
#in transformers, “pipeline” is very different from “model”. “Model” is the thing we download from the Hub, gpt2 in our case, which is, in fact, a valid PyTorch model with some additional restrictions and naming conventions introduced by the transformers framework. “Pipeline” is the object which runs the model under the hood to perform a certain high-level task, e.g. text-generation. The correspondence is not one-to-one, you can use various models for text-generation: gpt2, gtp2-medium, gpt2-large, fine-tuned GPT-2 versions, and custom user models. But you cannot use models with no generation capabilities, such as Bert, in this pipeline

#While pipelines are what Hugging Face newbies typically start with, for us, they are not very interesting. Pipelines perform a lot of steps under the hood, which are hard to understand and even harder to reproduce. They are hard to customize and totally useless for model training or fine-tuning, custom models, performing custom tasks, or in general, everything the developers in Hugging Face did not plan in advance. You only really know the transformers framework if you can do things in a pipeline-free way.


In [10]:
#text generation example without pipelines
model = transformers.GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)


In [11]:
#Neural networks are not able to work with raw text; they only understand numbers. We need a tokenizer to convert a text string into a list of numbers. But first, it breaks the string up into individual tokens, which most often means “words”, although some models can use word parts or even individual characters.
#Tokenization is a classical natural language processing task. Once the text is broken into tokens, each token is replaced by an integer number called encoding from a fixed dictionary. Note that a tokenizer, and especially its dictionary, is model-dependent:
#you cannot use Bert tokenizer with GPT-2, at least not unless you train the model from scratch. Some models, especially of the Bert family, like to use special tokens, such as [PAD], [CLS], [SEP], etc. GPT-2, in contrast, uses them very sparingly.
enc = tokenizer(['The elf queen'], return_tensors='pt')
#return_tensors=’pt’ option means returning PyTorch tensors; lists are returned otherwise
print('enc =', enc)
print(tokenizer.batch_decode(enc['input_ids']))
#The batch_decode() method decodes tokens back to the string “The elf queen”.


enc = {'input_ids': tensor([[  464, 23878, 16599]]), 'attention_mask': tensor([[1, 1, 1]])}
['The elf queen']


In [16]:
out = model.generate(input_ids=enc['input_ids'],
attention_mask=enc['attention_mask'], max_length=20)
#Generate the text using the generate() method of our model
print('out=', out)
print(tokenizer.batch_decode(out))
#then decode the new tokens.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


out= tensor([[  464, 23878, 16599,    11,   508,   550,   587,  4953,   329,   262,
         10325,   286,   262, 31959,    11,   550,   587,  4953,   329,   262]])
['The elf queen, who had been waiting for the arrival of the elves, had been waiting for the']


In [14]:
#But there is a problem in the code, how many number of times you run it result is same.
#Because the model that we have gotten without using pipelines is not having that randomness by default. But the pipelines tweaks the config to have randomness.

config = transformers.GPT2Config.from_pretrained(MODEL_NAME)
print(config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 50257
}



In [17]:
config.do_sample = \
          config.task_specific_params['text-generation']['do_sample']

config.max_length = \
          config.task_specific_params['text-generation']['max_length']

#The dict task_specific_params contains parameter adjustments for pipeline tasks, in this case text-generation. To activate these parameters, we copy them by hand to the object


In [19]:
out = model.generate(input_ids=enc['input_ids'],
attention_mask=enc['attention_mask'], max_length=20)
#Generate the text using the generate() method of our model
print('out=', out)
print(tokenizer.batch_decode(out))
#then decode the new tokens.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


out= tensor([[  464, 23878, 16599,    11,   508,   550,   587,  4953,   329,   262,
         10325,   286,   262, 31959,    11,   550,   587,  4953,   329,   262]])
['The elf queen, who had been waiting for the arrival of the elves, had been waiting for the']


In [22]:
# Still we dont have radomness because we have to create the model again with config as the parameter

model = transformers.GPT2LMHeadModel.from_pretrained(MODEL_NAME,config=config)
out = model.generate(input_ids=enc['input_ids'],
attention_mask=enc['attention_mask'], max_length=20)
#Generate the text using the generate() method of our model
print('out=', out)
print(tokenizer.batch_decode(out))



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


out= tensor([[  464, 23878, 16599,  3724,   287,   262,  2214,   286, 21682,  5536,
           379,   262,   886,   286,   262,  1175,    13,   632,  1718,  1295]])
['The elf queen died in the field of frost magic at the end of the war. It took place']
